<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

# _*Quantum algorithm for linear system of equations*_ 
***

***
### Contributors 
Shan Jin, Xi He, Xiaokai Hou, Li Sun, Dingding Wen, Shaojun Wu and Xiaoting Wang$^{1}$

1. Institute of Fundamental and Frontier Sciences, University of Electronic Science and Technology of China，Chengdu, China，610051
***

## Introduction
Solving linear equations is a very common problem in the fields of numerical optimization and machine learning. With the rapid expansion of data sets, solving linear equations becomes more and more difficult for classical computer. However, quantum computers can be used for solving linear systems of algebraic equations with exponential speed up compared with classical computers. Therefore, the use of quantum computers to solve the system of linear equations can greatly reduce the computational complexity and time complexity. 

#### Experiment design
<p>We have a Hermitian $N \times N$ matrix $A$, and a unit vector $\vec b$, suppose we would like to find $x$ satisfying $A\vec x = \vec b$.[[1]](#cite_1) <br />(1)Represent $b$ as a quantum state $|b\rangle = \sum_{i=1}^{N} b_i |i\rangle$.<br />(2) Apply the conditional Hamiltonian evolution $e^{iAt}$ to $|b\rangle$ for a superposition of different times $t$. With the phase estimation algorithm, we can decompose $|b\rangle$ in the eigenbasis of $A$ and to find the corresponding eigenvalues $\lambda_j$. After this stage, the state of the system is close to $\sum_{j=1}^{N} \beta_j |u_j\rangle |\lambda_j\rangle$, where $u_j$ is the eigenvector basis of $A$, and $|b\rangle = \sum_{j=1}^{N}\beta_j |u_j\rangle$. <br />(3)Uncompute the $|\lambda_j\rangle$ register and we get a state which is propotional to $\sum_{j=1}^{N} \beta_j \lambda_{j}^{-1}|u_j\rangle = A^{-1}|b\rangle = |x\rangle$.</p>
The schematic diagram of quantum K-Means is the following picture.[[2]](#cite_2) And to make our algorithm can be run using qiskit, we design a more detailed circuit to achieve our algorithm in the next section. 
<img src="../images/hhl_1.jpg" width="400 px">
#### Quantum K-Means circuit
<img src="../images/hhl_2.png">
#### Parameter declaration
Here we set the parameters as follwing, the aim is to ensure the precision.[[3]](#cite_3) <br />
$r = 4$, $t_0=2\pi$, $S = \begin{pmatrix}1 & 0 \\ 0 & i\end{pmatrix}$, $H = \frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1 \\ 1 & -1\end{pmatrix}$, 
$R_y(\theta) = \begin{pmatrix}cos(\theta/2) & -sin(\theta/2) \\ sin(\theta/2) & cos(\theta/2)\end{pmatrix}$


## Data declaration
The aim of this algorithm is to solve the linear system of equations. As a demo, we take a linear equation as $A \vec x = \vec b$. In this equition, for example, we define the matrix $A$ as $A = \begin{pmatrix}2 & -1 \\ -1 & 2\end{pmatrix}$ and $\vec b$ as $\vec b=\begin{pmatrix}0 \\ 1\end{pmatrix}$. After making the $A$ and $\vec b$ as input, we implement the algorithm to obtain the result vector $\vec x$.

## Quantum algorithm for linear system of equations program

In [1]:
# Checking the version of PYTHON; we only support > 3.5
import sys
if sys.version_info < (3,5):
    raise Exception('Please use Python version 3.5 or greater.')

from qiskit import QuantumProgram
import math
#import Qconfig
#Define a QuantumProgram object 
Q_program = QuantumProgram()
#Q_program.set_api(Qconfig.APItoken, Qconfig.config['url'])
pi = math.pi


In this section, we first judge the version of Python and import the packages of qiskit, math to implement the following code. We show our algorithm on the ibm_qasm_simulator, if you need to run it on the real quantum conputer, please remove the "#" in frint of "import Qconfig" and the set_api() function.

In [2]:
def solve_linear_sys():  

   #Q_program.set_api(Qconfig.APItoken, Qconfig.config['url'])     
        
    # create Quantum Register called "qr" with 4 qubits
    qr = Q_program.create_quantum_register("qr", 4)
    # create Quantum Register called "cr" with 4 qubits
    cr = Q_program.create_classical_register("cr", 4)
    # Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
    # and your Classical Register "cr"
    qc = Q_program.create_circuit("solve_linear_sys", [qr], [cr])        
    
    # Initialize times that we get the result vector 
    n0 = 0
    n1 = 0
    
    for i in range(10):
        #Set the input|b> state"
        qc.x(qr[2])
        
        #Set the phase estimation circuit
        qc.h(qr[0])
        qc.h(qr[1]) 
        qc.u1(pi, qr[0])
        qc.u1(pi/2, qr[1])
        qc.cx(qr[1], qr[2])
    
        #The quantum inverse  Fourier transform 
        qc.h(qr[0])
        qc.cu1(-pi/2, qr[0], qr[1])
        qc.h(qr[1])
    
        #R（lamda^-1） Rotation
        qc.x(qr[1])
        qc.cu3(pi/16, 0, 0, qr[0], qr[3])
        qc.cu3(pi/8, 0, 0, qr[1], qr[3])   
    
        #Uncomputation
        qc.x(qr[1])
        qc.h(qr[1])
        qc.cu1(pi/2, qr[0], qr[1])
        qc.h(qr[0])
    
        qc.cx(qr[1], qr[2])
        qc.u1(-pi/2, qr[1])
        qc.u1(-pi, qr[0])
    
        qc.h(qr[1]) 
        qc.h(qr[0])
         
        # To measure the whole quantum register
        qc.measure(qr[0], cr[0])
        qc.measure(qr[1], cr[1])
        qc.measure(qr[2], cr[2])
        qc.measure(qr[3], cr[3])

        result = Q_program.execute("solve_linear_sys", shots=8192, backend='local_qasm_simulator')   
        
        # Get the sum og all results
        n0 = n0 + result.get_data("solve_linear_sys")['counts']['1000']
        n1 = n1 + result.get_data("solve_linear_sys")['counts']['1100']
    
        # print the result
        print(result)
        print(result.get_data("solve_linear_sys"))
        
                
        # Reset the circuit
        qc.reset(qr)
    
    # calculate the scale of the elements in result vectot and print it.
    p = n0/n1
    print(n0)
    print(n1)
    print(p)


# The test function
if __name__ == "__main__":
    solve_linear_sys()


COMPLETED
{'counts': {'0100': 7801, '0000': 2, '1100': 318, '1000': 71}, 'time_taken': 0.395833799}
COMPLETED
{'counts': {'0100': 7797, '0000': 1, '1100': 316, '1000': 78}, 'time_taken': 0.647315894}
COMPLETED
{'counts': {'0100': 7818, '0000': 3, '1100': 300, '1000': 71}, 'time_taken': 0.883254109}
COMPLETED
{'counts': {'0100': 7767, '0000': 3, '1100': 332, '1000': 90}, 'time_taken': 1.189058497}
COMPLETED
{'counts': {'0100': 7770, '0000': 2, '1100': 329, '1000': 91}, 'time_taken': 1.480844052}
COMPLETED
{'counts': {'0100': 7790, '0000': 2, '1100': 340, '1000': 60}, 'time_taken': 1.908946923}
COMPLETED
{'counts': {'0100': 7801, '0000': 4, '1100': 317, '1000': 70}, 'time_taken': 2.065915418}
COMPLETED
{'counts': {'0100': 7794, '0000': 4, '1100': 307, '1000': 87}, 'time_taken': 2.424704692}
COMPLETED
{'counts': {'0100': 7776, '0000': 2, '1100': 332, '1000': 82}, 'time_taken': 2.729468201}
COMPLETED
{'counts': {'0100': 7785, '0000': 2, '1100': 335, '1000': 70}, 'time_taken': 4.292395966}


## Result analysis
According to the result, we can obtain the scale of two elements in $\vec x$. Via features of linear system, we can get
$A\vec x=A\begin{pmatrix}a_1 \\ ka_2\end{pmatrix}=\vec b=\begin{pmatrix}0 \\ 1\end{pmatrix}$. So we can get the answer, $\vec x=\begin{pmatrix}0.32665 \\0.67335\end{pmatrix} $.<br \> For more examples, we test different value of $\vec b$ and get the answer about $\vec x $ as following table: <br \>
<img src="../images/hhl_3.png" width="500 px">
And the statistic result is showed as following:
<img src="../images/hhl_4.png" width="500 px">

## Reference
<cite>[1]Quantum Algorithm for Linear Systems of Equations(see [Aram W. Harrow, Avinatan Hassidim, and Seth Lloyd
Phys. Rev. Lett. 103, 150502](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502))</cite><a id='cite_1'></a><br \>
<cite>[2]Quantum circuit design for solving linear systems of equations(see [Yudong Cao,Anmer Daskin,Steven Frankel & Sabre Kais](https://www.tandfonline.com/doi/abs/10.1080/00268976.2012.668289))</cite><a id='cite_2'></a><br \>
<cite>[3]Experimental realization of quantum algorithm for solving linear systems of equations(see [Jian Pan, Yudong Cao, Xiwei Yao, Zhaokai Li, Chenyong Ju, Hongwei Chen, Xinhua Peng, Sabre Kais, and Jiangfeng Du Phys. Rev. A 89, 022313](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.89.022313))</cite><a id='cite_3'></a>

In [5]:
%run "../version.ipynb"

Package,Version
QISKit,0.4.8
IBMQuantumExperience,>= 1.8.26
numpy,">= 1.13, < 1.14"
scipy,">= 0.19, < 0.20"
matplotlib,">= 2.0, < 2.1"
